In [ ]:
from google.colab import files
uploaded = files.upload()


In [19]:
from google.colab import files
uploaded = files.upload()

Saving test1.zip to test1.zip


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
import itertools
import copy
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [4]:
transform_cifar = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_cifar = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
test_cifar  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)

train_loader_cifar = DataLoader(train_cifar, batch_size=64, shuffle=True)
test_loader_cifar  = DataLoader(test_cifar, batch_size=64, shuffle=False)


100%|██████████| 170M/170M [00:04<00:00, 42.3MB/s]


In [23]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform_dogs = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_dogs = datasets.ImageFolder('/content/train_data', transform=transform_dogs)
test_dogs  = datasets.ImageFolder('/content/test_data', transform=transform_dogs)

train_loader_dogs = DataLoader(train_dogs, batch_size=32, shuffle=True)
test_loader_dogs  = DataLoader(test_dogs, batch_size=32, shuffle=False)

print("Classes:", train_dogs.classes)


Classes: ['train']


In [5]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes, activation):
        super(CustomCNN, self).__init__()

        if activation == "relu":
            act = nn.ReLU()
        elif activation == "tanh":
            act = nn.Tanh()
        else:
            act = nn.LeakyReLU()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            act,
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            act,
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            act,
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*8*8, 256),
            act,
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [6]:
def init_weights(model, init_type):
    for m in model.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            if init_type == "xavier":
                nn.init.xavier_uniform_(m.weight)
            elif init_type == "kaiming":
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
            else:
                nn.init.normal_(m.weight, mean=0.0, std=0.02)


In [7]:
def train_model(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0
    correct = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

    return running_loss/len(loader), correct/len(loader.dataset)


def eval_model(model, loader, criterion):
    model.eval()
    running_loss = 0
    correct = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

    return running_loss/len(loader), correct/len(loader.dataset)


In [8]:
activations = ["relu", "tanh", "leaky"]
inits = ["xavier", "kaiming", "random"]
optimizers_list = ["sgd", "adam", "rmsprop"]

criterion = nn.CrossEntropyLoss()

best_models = {}

for act, init, opt in itertools.product(activations, inits, optimizers_list):
    print(f"\nConfig: {act} | {init} | {opt}")

    model = CustomCNN(num_classes=10, activation=act).to(device)
    init_weights(model, init)

    if opt == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=0.01)
    elif opt == "adam":
        optimizer = optim.Adam(model.parameters(), lr=0.001)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=0.001)

    for epoch in range(5):
        train_loss, train_acc = train_model(model, train_loader_cifar, criterion, optimizer)
        val_loss, val_acc = eval_model(model, test_loader_cifar, criterion)
        print(f"Epoch {epoch+1}: Val Acc={val_acc:.4f}")

    best_models[f"{act}_{init}_{opt}"] = copy.deepcopy(model.state_dict())



Config: relu | xavier | sgd
Epoch 1: Val Acc=0.5056
Epoch 2: Val Acc=0.5268
Epoch 3: Val Acc=0.6094
Epoch 4: Val Acc=0.6426
Epoch 5: Val Acc=0.6669

Config: relu | xavier | adam
Epoch 1: Val Acc=0.4674
Epoch 2: Val Acc=0.5309
Epoch 3: Val Acc=0.5841
Epoch 4: Val Acc=0.6229
Epoch 5: Val Acc=0.6018

Config: relu | xavier | rmsprop
Epoch 1: Val Acc=0.4357
Epoch 2: Val Acc=0.4774
Epoch 3: Val Acc=0.5838
Epoch 4: Val Acc=0.5831
Epoch 5: Val Acc=0.5769

Config: relu | kaiming | sgd
Epoch 1: Val Acc=0.5176
Epoch 2: Val Acc=0.5520
Epoch 3: Val Acc=0.5880
Epoch 4: Val Acc=0.6071
Epoch 5: Val Acc=0.6216

Config: relu | kaiming | adam
Epoch 1: Val Acc=0.4176
Epoch 2: Val Acc=0.4876
Epoch 3: Val Acc=0.4961
Epoch 4: Val Acc=0.4933
Epoch 5: Val Acc=0.5663

Config: relu | kaiming | rmsprop
Epoch 1: Val Acc=0.3607
Epoch 2: Val Acc=0.4586
Epoch 3: Val Acc=0.5188
Epoch 4: Val Acc=0.5702
Epoch 5: Val Acc=0.6232

Config: relu | random | sgd
Epoch 1: Val Acc=0.4813
Epoch 2: Val Acc=0.5818
Epoch 3: Val Acc

In [9]:
resnet = models.resnet18(pretrained=True)

for param in resnet.parameters():
    param.requires_grad = False

resnet.fc = nn.Linear(resnet.fc.in_features, 10)
resnet = resnet.to(device)

optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)

for epoch in range(5):
    train_loss, train_acc = train_model(resnet, train_loader_cifar, criterion, optimizer)
    val_loss, val_acc = eval_model(resnet, test_loader_cifar, criterion)
    print(f"ResNet Epoch {epoch+1} Acc={val_acc:.4f}")


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 183MB/s]


ResNet Epoch 1 Acc=0.6287
ResNet Epoch 2 Acc=0.6341
ResNet Epoch 3 Acc=0.6432
ResNet Epoch 4 Acc=0.6516
ResNet Epoch 5 Acc=0.6521
